#### ĐỌC ẢNH TO THEO CỬA SỔ NHỎ RỒI LƯU LẠI THÀNH 1 ẢNH NHƯ BAN ĐẦU

In [ ]:

import rasterio
import numpy as np
import rasterio
import argparse
from rasterio.windows import Window
import threading
from tqdm import tqdm

import tensorflow as tf
# import Vectorization


import os, glob
num_bands = 4
path_image = '/home/skymap/data/Bahrain_change/set19/T19_rgb/T19_54/tif'
dir_out = '/home/skymap/data/Bahrain_change/set19/test_h/b.tif'
size = ""
with rasterio.open(path_image) as raster:
    meta = raster.meta
    meta.update({'count': 1, 'nodata': 0,"dtype":"uint8"})
    height, width = raster.height, raster.width
    input_size = size
    stride_size = input_size - input_size //4
    padding = int((input_size - stride_size) / 2)

    list_coordinates = []
    for start_y in range(0, height, stride_size):
        for start_x in range(0, width, stride_size):
            x_off = start_x if start_x==0 else start_x - padding
            y_off = start_y if start_y==0 else start_y - padding

            end_x = min(start_x + stride_size + padding, width)
            end_y = min(start_y + stride_size + padding, height)

            x_count = end_x - x_off
            y_count = end_y - y_off
            list_coordinates.append(tuple([x_off, y_off, x_count, y_count, start_x, start_y]))
    with rasterio.open(dir_out,'w+', **meta, compress='lzw') as r:
        read_lock = threading.Lock()
        write_lock = threading.Lock()

        def process(coordinates):
            x_off, y_off, x_count, y_count, start_x, start_y = coordinates
            read_wd = Window(x_off, y_off, x_count, y_count)
            with read_lock:
                values = raster.read(window=read_wd)[0:num_bands]
            if raster.profile["dtype"]=="uint8":
                # print('vao')
                image_detect = values.transpose(1,2,0).astype(int)
                
        
            img_temp = np.zeros((input_size, input_size, image_detect.shape[2]))
            mask = np.pad(np.ones((stride_size, stride_size), dtype=np.uint8), ((padding, padding),(padding, padding)))
            shape = (stride_size, stride_size)
            if y_count < input_size or x_count < input_size:
                img_temp = np.zeros((input_size, input_size, image_detect.shape[2]))
                mask = np.zeros((input_size, input_size), dtype=np.uint8)
                if start_x == 0 and start_y == 0:
                    img_temp[(input_size - y_count):input_size, (input_size - x_count):input_size] = image_detect
                    mask[(input_size - y_count):input_size-padding, (input_size - x_count):input_size-padding] = 1
                    shape = (y_count-padding, x_count-padding)
                elif start_x == 0:
                    img_temp[0:y_count, (input_size - x_count):input_size] = image_detect
                    if y_count == input_size:
                        mask[padding:y_count-padding, (input_size - x_count):input_size-padding] = 1
                        shape = (y_count-2*padding, x_count-padding)
                    else:
                        mask[padding:y_count, (input_size - x_count):input_size-padding] = 1
                        shape = (y_count-padding, x_count-padding)
                elif start_y == 0:
                    img_temp[(input_size - y_count):input_size, 0:x_count] = image_detect
                    if x_count == input_size:
                        mask[(input_size - y_count):input_size-padding, padding:x_count-padding] = 1
                        shape = (y_count-padding, x_count-2*padding)
                    else:
                        mask[(input_size - y_count):input_size-padding, padding:x_count] = 1
                        shape = (y_count-padding, x_count-padding)
                else:
                    img_temp[0:y_count, 0:x_count] = image_detect
                    mask[padding:y_count, padding:x_count] = 1
                    shape = (y_count-padding, x_count-padding)

                image_detect = img_temp
            #print('image',image_detect.shape)
            mask = (mask!=0)
            
            # print(image_detect.shape, 'eeeee')

            if np.count_nonzero(image_detect) > 0:
                if len(np.unique(image_detect)) <= 2:
                    pass
                else:
                    #xuli ....

                    with write_lock:
                        r.write(image_detect, window=Window(start_x, start_y, shape[1], shape[0]))
        with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
            results = list(tqdm(executor.map(process, list_coordinates), total=len(list_coordinates)))

### CẮT ẢNH RA 1 FOLDER CON


In [2]:
import rasterio.mask
import rasterio
from rasterio import windows
from itertools import product
import numpy as np
import glob, os
from tqdm import tqdm


def get_tiles(ds, width, height, stride):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, stride), range(0, nrows, stride))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    offset = []
    for col_off, row_off in offsets:
        if row_off + width > nrows:
            row_off = nrows - width
        if  col_off + height > nols:
            col_off = nols - height
        offset.append((col_off, row_off))
    offset = set(offset)
    for col_off, row_off in tqdm(offset): 
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform

In [5]:
path_img = '/home/skymap/data/Bahrain_change/set19/T19_rgb/T19_54.tif'
out_dir = '/home/skymap/data/Bahrain_change/set19/test_h'
name = os.path.basename(path_img).replace('.tif','')
with rasterio.open(path_img) as inds:
  
    tile_width, tile_height = 512, 512
    stride = 512
    height = inds.height
    width = inds.width
    
    i = 0
    for window, transform in get_tiles(inds, tile_width, tile_height, stride):
        if np.random.random_sample()>0.2:
        
            outpath_image = os.path.join(out_dir, f'{name}_box_2_{i}'+".tif")
        else:
            
            outpath_image = os.path.join(out_dir, f'{name}_box_2_{i}'+".tif")
        img = inds.read(window=window)
    
        if np.count_nonzero(img):

        #write image
            out_meta = inds.meta
            out_meta.update({"driver": "GTiff",
                    "height": img.shape[1],
                    "width": img.shape[2],
                    "transform": transform,
                    "nodata": 0})
            with rasterio.open(outpath_image, "w", compress='lzw', **out_meta) as dest:
                dest.write(img)

        i+=1  
       

  0%|          | 0/640 [00:00<?, ?it/s]

100%|██████████| 640/640 [00:37<00:00, 16.96it/s]
